In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm


In [ ]:
# ───────────────────────────────────────────────
# 1.  Parametry globalne i badana funkcja
# ───────────────────────────────────────────────
A = -2 * math.pi        # lewa granica
B =  4 * math.pi        # prawa granica
L = B - A               # długość okresu (6π)
ω = 2 * math.pi / L     # podstawowa pulsacja

def f(x: np.ndarray) -> np.ndarray:
    """f(x) = e^(−3 sin x) + 3 cos x"""
    return np.exp(-3 * np.sin(x)) + 3 * np.cos(x)

def uniform_nodes(a: float, b: float, n: int) -> np.ndarray:
    return np.linspace(a, b, n)

# ───────────────────────────────────────────────────────────
# 1.  Współczynniki a_k, b_k
# ───────────────────────────────────────────────────────────
def coeffs_uniform(xs: np.ndarray,
                           ys: np.ndarray,
                           m: int):
    n = len(xs)
    if len(ys) != n or not (1 <= m <= (n-1)//2):
        raise ValueError("warunek 2m < n nie spełniony")

    x_t = (xs - xs[0]) * (2*math.pi)/(xs[-1]-xs[0]) - math.pi
    k   = np.arange(1, m+1)

    cos_kx = np.cos(np.outer(x_t, k))
    sin_kx = np.sin(np.outer(x_t, k))

    a0 = 2/n * np.sum(ys)
    ak = 2/n * (ys @ cos_kx)
    bk = 2/n * (ys @ sin_kx)
    return np.concatenate(([a0], ak)), bk          # (m+1,), (m,)

def make_trig_poly(a: np.ndarray, b: np.ndarray, a_orig: float, b_orig: float):
    m     = len(b)
    scale = (2*math.pi)/(b_orig - a_orig)
    def Pm(x):
        x_t = (x - a_orig)*scale - math.pi
        s   = 0.5*a[0]
        for k in range(1, m+1):
            s += a[k]*np.cos(k*x_t) + b[k-1]*np.sin(k*x_t)
        return s
    return np.vectorize(Pm)

def calc_err(true_f, approx_f, xs):
    diff = np.abs(true_f(xs) - approx_f(xs))
    return diff.max(), np.sqrt((diff**2).sum()) / len(diff)

In [ ]:
# ───────────────────── 3.  Skan po (n,m) + CSV ───────────────────────────
x_eval = np.linspace(A, B, 5_000)
results = []                                  # (n, m, max, avg)

for n in range(3, 101, 2):      
    print(n,end=",")# 3,5,…,101
    xs = uniform_nodes(A, B, n)
    ys = f(xs)
    for m in range(1, (n-1)//2):
        a, b = coeffs_uniform(xs, ys, m)
        Pm   = make_trig_poly(a, b, A, B)
        max_e, avg_e = calc_err(f, Pm, x_eval)
        results.append((n, m, max_e, avg_e))

df = pd.DataFrame(results, columns=["n", "m", "max_error", "avg_error"])
df.to_csv("errors_trig_scan.csv", index=False)
print(f"✓ zapisano {len(df)} wierszy do errors_trig_scan.csv")




In [ ]:
df = pd.read_csv("errors_trig_scan.csv")
from pathlib import Path
# ───────────────────── 4.  Najlepsza / najgorsza aproksymacja ────────────
best_row  = df.loc[df.avg_error.idxmin()]
worst_row = df.loc[df.avg_error.idxmax()]

def plot_comparison(n: int, m: int, title: str,
                    x_eval: np.ndarray = np.linspace(A, B, 5_000),
                    show: bool = True,
                    save: bool = False,
                    folder: str = "plots_compare"):

    # --- 1. weryfikacja 2m < n ------------------------------------------------
    if not 2*m < n:
        raise ValueError("musi być 2m < n")

    # --- 2. ścieżka pliku ------------------------------------------------------
    if save:
        out_path = Path(folder) / f"{title.replace(' ','_')}_n{n:03d}_m{m:03d}.png"
        if out_path.exists():
            return
        out_path.parent.mkdir(parents=True, exist_ok=True)

    # --- 3. dane ---------------------------------------------------------------
    xs = uniform_nodes(A, B, n)
    ys = f(xs)
    a, b = coeffs_uniform(xs, ys, m)
    Pm   = make_trig_poly(a, b, A, B)

    max_e, avg_e = calc_err(f, Pm, x_eval)

    # --- 4. wykres -------------------------------------------------------------
    fig, ax = plt.subplots(2, 1, figsize=(9, 8),
                           sharex=True,
                           gridspec_kw={"height_ratios": [3, 1]})
    fig.suptitle(f"{title}   (n={n}, m={m})\n"
                 f"max={max_e:.2e}   średni={avg_e:.2e}",
                 fontsize=13)

    # górny panel: funkcja + aproksymacja
    ax[0].plot(x_eval, f(x_eval),  label="f(x)")
    ax[0].plot(x_eval, Pm(x_eval), label=f"P₍{m}₎(x)")
    ax[0].scatter(xs, ys, s=10, color="black", label="węzły")
    ax[0].grid(); ax[0].legend()

    # dolny panel: błąd punktowy
    diff = np.abs(f(x_eval) - Pm(x_eval))
    ax[1].plot(x_eval, diff, color="tab:red")
    ax[1].set_ylabel("|f−P|"); ax[1].set_xlabel("x"); ax[1].grid()

    fig.tight_layout(rect=[0, 0, 1, 0.95])

    # --- 5. zapis / wyświetlenie ----------------------------------------------
    if save:
        fig.savefig(out_path, dpi=200)
        print("✓ zapisano", out_path)
    if show:
        plt.show()
    else:
        plt.close(fig)

print("\nNajlepsza aproksymacja:")
print(best_row)
plot_comparison(int(best_row.n), int(best_row.m), "Najlepsza")

print("\nNajgorsza aproksymacja:")
print(worst_row)
plot_comparison(int(worst_row.n), int(worst_row.m), "Najgorsza")

# ───────────────────── 5.  Heat-mapy Seaborn ─────────────────────────────
pivot_max = df.pivot(index="n", columns="m", values="max_error")
pivot_avg = df.pivot(index="n", columns="m", values="avg_error")

for data, title, cmap, label in [
    (pivot_max, "Błąd maksymalny", "coolwarm", "max_error"),
    (pivot_avg, "Błąd średni", "YlGnBu", "avg_error")
]:
    plt.figure(figsize=(14,10))
    sns.heatmap(data, cmap=cmap, annot=False, cbar_kws={"label": label})
    plt.title(f"{title} (skala liniowa)")
    plt.xlabel("rząd m"); plt.ylabel("liczba węzłów n")
    plt.tight_layout(); plt.show()


cut_max = pivot_max.loc[pivot_max.index<=31, pivot_max.columns<=15]
cut_avg = pivot_avg.loc[pivot_avg.index<=31, pivot_avg.columns<=15]

for data, title, cmap, label in [
    (cut_max, "Błąd maksymalny (n≤25,m≤12)", "coolwarm", "max_error"),
    (cut_avg, "Błąd średni (n≤25,m≤12)",    "YlGnBu",   "avg_error")
]:
    plt.figure(figsize=(10,6))
    sns.heatmap(data, cmap=cmap, annot=False, cbar_kws={"label": label})
    plt.title(title)
    plt.xlabel("rząd m"); plt.ylabel("liczba węzłów n")
    plt.tight_layout(); plt.show()

In [ ]:

from pathlib import Path
def plot_nm(n: int, m: int,
                    x_eval: np.ndarray = np.linspace(A, B, 5_000),
                    show: bool = True,
                    save: bool = False,
                    folder: str = "plots"):

    if not 2*m < n:
        raise ValueError("musi być 2m < n")

    # pomiń, jeśli obrazek istnieje
    if save:
        out_path = Path(folder) / f"approxTrig_n{n:03d}_m{m:03d}.png"
        if out_path.exists():
            return
        out_path.parent.mkdir(parents=True, exist_ok=True)

    xs = uniform_nodes(A, B, n)
    ys = f(xs)
    a, b = coeffs_uniform(xs, ys, m)
    Pm   = make_trig_poly(a, b, A, B)

    max_e, avg_e = calc_err(f, Pm, x_eval)

    # --- wykresy ---
    fig, ax = plt.subplots(2, 1, figsize=(9, 8),
                           sharex=True,
                           gridspec_kw={"height_ratios":[3,1]})
    fig.suptitle(f"Aproksymacja trygonometryczna  n={n}, m={m}\n"
                 f"max={max_e:.2e}   średni={avg_e:.2e}", fontsize=13)

    ax[0].plot(x_eval, f(x_eval),  label="f(x)")
    ax[0].plot(x_eval, Pm(x_eval), label=f"P₍{m}₎(x)")
    ax[0].scatter(xs, ys, s=10, color="black", label="węzły")
    ax[0].grid(); ax[0].legend()

    diff = np.abs(f(x_eval) - Pm(x_eval))
    ax[1].plot(x_eval, diff, color="tab:red")
    ax[1].set_ylabel("|f−P|"); ax[1].set_xlabel("x"); ax[1].grid()

    fig.tight_layout(rect=[0,0,1,0.95])

    if save:
        fig.savefig(out_path, dpi=200)
        print("✓ zapisano", out_path)
    if show:
        plt.show()
    else:
        plt.close(fig)

# ─────────────────  MAIN  ────────────────────────────────
if __name__ == "__main__":
    # pojedyncza demonstracja
    plot_nm(51, 12)

    # cała seria (bez wyświetlania, tylko zapis)
    make_all = True
    if make_all:
        for n in range(3, 101, 2):          # 3,5,…,101
            for m in range(1, (n-1)//2):
                plot_nm(n, m, show=False, save=True)
        print("✓ wszystkie wykresy w katalogu  plots/")

In [ ]:
def plot_multi_m(n: int, m_list: list[int],
                 x_eval: np.ndarray = np.linspace(A, B, 5_000),
                 show: bool = True,
                 save: bool = False,
                 folder: str = "plots_multi"):
    bad = [m for m in m_list if not (1 <= m <= (n-1)//2)]
    if bad:
        raise ValueError(f"dla n={n} niepoprawne m: {bad}")

    out_path = (Path(folder) /
                f"multi_n{n:03d}_{'-'.join(map(str,m_list))}.png")
    if save:
        if out_path.exists():
            return
        out_path.parent.mkdir(parents=True, exist_ok=True)

    xs = uniform_nodes(A, B, n)
    ys = f(xs)
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(x_eval, f(x_eval), label="f(x)", linewidth=2)

    cols = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    for i, m in enumerate(sorted(set(m_list))):
        a, b = coeffs_uniform(xs, ys, m)
        Pm   = make_trig_poly(a, b, A, B)
        ax.plot(x_eval, Pm(x_eval),
                linestyle="--", color=cols[i % len(cols)],
                label=f"P_{m}(x)")
    ax.scatter(xs, ys, s=12, color="black", label="węzły")
    ax.set_title(f"Aproksymacje: n={n},  m={m_list}")
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.grid(); ax.legend()

    if save:
        fig.savefig(out_path, dpi=200); print("✓ zapisano", out_path)
    if show: plt.show()
    else:    plt.close(fig)
        
if __name__ == "__main__":
    plot_multi_m(51, [3, 4, 5])

    examples = {
        9:  [3, 4],                # max m = 4
        15: [5, 6, 7],             # max m = 7
        21: [8, 9, 10],            # max m = 10
        25: [9, 10, 11, 12],       # max m = 12
        31: [12, 13, 14, 15],       # max m = 15
        45: [5, 10, 15, 20],      # max m = 22
        51: [5, 10, 15, 20, 25],  # max m = 25
        60: [5, 10, 15, 20, 25, 29],  # max m = 29
        75: [10, 15, 20, 25, 30, 35], # max m = 37
        87: [10, 20, 30, 40],     # max m = 43
    }
    for n_ex, ms in examples.items():
        plot_multi_m(n_ex, ms, show=False, save=True)

In [ ]:
def plot_multi_n(m: int, n_list: list[int],
                 x_eval: np.ndarray = np.linspace(A, B, 5_000),
                 show: bool = True,
                 save: bool = False,
                 folder: str = "plots_multi_n"):

    # ── 1. poprawiony filtr 2m < n  ──────────────────────────
    bad = [n for n in n_list if not (2*m < n)]
    if bad:
        raise ValueError(f"przy m={m} niepoprawne n: {bad}")

    # ── przygotowanie pliku ──────────────────────────────────
    out_path = Path(folder) / f"multi_m{m:03d}_{'-'.join(map(str,n_list))}.png"
    if save and out_path.exists():
        return
    out_path.parent.mkdir(parents=True, exist_ok=True)

    # ── wykres ───────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(x_eval, f(x_eval), label="f(x)", linewidth=2)

    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    for i, n in enumerate(sorted(set(n_list))):
        xs = uniform_nodes(A, B, n)
        a, b = coeffs_uniform(xs, f(xs), m)
        Pm   = make_trig_poly(a, b, A, B)
        ax.plot(x_eval, Pm(x_eval),
                linestyle="--", color=colors[i % len(colors)],
                label=f"P{m}(x) | n={n}")
        if n == min(n_list):
            ax.scatter(xs, f(xs), s=12, color="black", label=f"węzły (n={n})")

    ax.set_title(f"Aproksymacje: m={m},  n={n_list}")
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.grid(); ax.legend()

    if save:
        fig.savefig(out_path, dpi=200); print("✓ zapisano", out_path)
    if show:
        plt.show()
    else:
        plt.close(fig)


In [ ]:
if __name__ == "__main__":
    # jeden wykres na ekran:
    plot_multi_n(5, [13, 25, 41])

    # seria wykresów tylko do plików:
    examples_m = {
    3:  [9, 13, 17, 21, 25],        # możliwe n > 6
    5:  [13, 17, 21, 25, 29, 33],   # możliwe n > 10
    8:  [19, 23, 27, 31, 35],       # możliwe n > 16
    10: [23, 27, 31, 35, 39],       # możliwe n > 20
    14: [29, 33, 37, 41, 45, 49],    # możliwe n > 28
    20: [60, 75, 87],                      # n > 40
    25: [60, 75, 87],                      # n > 50
    30: [75, 87],                          # n > 60
    35: [75, 87],                          # n > 70
    40: [87,96]                               # n > 80 
    }
    for m_ex, ns in examples_m.items():
        plot_multi_n(m_ex, ns, show=False, save=True)

In [ ]:
# ─────────────────────────────────────────────────────────────
# seria wykresów (tylko zapis do plików) – stałe m = 3, 6, 8, 9
# ─────────────────────────────────────────────────────────────
examples_m = {
    3: [9, 13, 17, 21, 25],                 # n >  6
    6: [13, 17, 21, 25, 29, 33],            # n > 12
    8: [19, 23, 27, 31, 35],                # n > 16
    9: [19, 23, 27, 31, 35]                 # n > 18
}

for m_ex, ns in examples_m.items():
    plot_multi_n(m_ex, ns, show=False, save=True)
print("✓ zapisano zestaw wykresów multi-n dla m = 3, 6, 8, 9")